In [ ]:
import numpy as np
import os
import pretty_midi
import sys
import librosa
import matplotlib.pyplot as plt
import keras
import tensorflow as tf
from keras.utils import pad_sequences
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential, Model
from keras.layers import LSTM, Dense, Input, Dropout
from tensorflow.keras.mixed_precision import LossScaleOptimizer
data_dir_X = "D:/PA/numpy/"
data_dir_Y = "D:/PA/midi/" 
file_list_X = os.listdir(data_dir_X)
file_list_Y = os.listdir(data_dir_Y)
file_list_numpy = os.listdir(data_dir_X)  # list all files in the directory
file_list_midi = os.listdir(data_dir_Y)  # list all files in the directory

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
  except RuntimeError as e:
    print(e)

In [ ]:
def generator(file_list_numpy, file_list_midi, batch_size):
    
    #fill batch
    curIndex = 0
    batch_x = np.zeros((batch_size,160000))
    batch_y = np.zeros((batch_size,2502))
    while True:

        for i in range(batch_size):
            
            #get X
            file_path = os.path.join(data_dir_X, file_list_numpy[curIndex])
            x_train = np.load(file_path).flatten()
            
            
            #get Y
            midi_data = pretty_midi.PrettyMIDI(data_dir_Y + file_list_midi[curIndex])
            instr_notes = []
            y = []
            y_train = []
            
            for instr in midi_data.instruments:
                for note in instr.notes:
                    instr_notes.append((instr.name, pretty_midi.note_number_to_name(note.pitch), note))


            instr_notes.sort(key=lambda tup: (tup[2].start, tup[2].end))
            for j, note in enumerate(instr_notes):
                y.append(round(note[2].start, 3))
                y.append(round(note[2].end, 3))
                y.append(note[2].pitch)
            y_train.append(y)

            y_train = np.array(y_train)
            
            normalized_y_train = []
            padded_y_train = pad_sequences(y_train, 2502, padding='post', dtype='float')
            normalized_y_train = np.array(padded_y_train)
            
            batch_x[i] = x_train
            batch_y[i] = normalized_y_train[0]
            
            if i == len(file_list_numpy)-2:
                i = 0
                
            if curIndex == len(file_list_numpy)-2:
                curIndex = 0
                
            curIndex += 1
            
        
        yield batch_x.reshape( batch_size, -1, batch_x.shape[1]), batch_y.reshape( batch_size, -1, batch_y.shape[1])

In [ ]:
#normalized_y_train = normalized_y_train.reshape(normalized_y_train.shape[0], normalized_y_train.shape[2])

input_shape = (1, 160000)
inputs = Input(shape=input_shape)

lstm_out = LSTM(256, return_sequences=True, return_state=True)(inputs) # return_state return 3 outputs
lstm_out = LSTM(256, return_sequences=True, return_state=True)(lstm_out)
lstm_out = LSTM(256, return_sequences=True, return_state=True)(lstm_out)
lstm_out = LSTM(256, return_sequences=True, return_state=True)(lstm_out)
lstm_out = LSTM(256, return_sequences=True, return_state=True)(lstm_out)
lstm_out = LSTM(256, return_sequences=True, return_state=True)(lstm_out)
lstm_out = LSTM(256, return_sequences=True, return_state=True)(lstm_out)
lstm_out = LSTM(256, return_sequences=True, return_state=True)(lstm_out)
lstm_out = LSTM(256, return_sequences=True, return_state=True)(lstm_out)
lstm_out = LSTM(256, return_sequences=True)(lstm_out) # return output gate only
output = Dense(2501, activation='relu')(lstm_out)

model = Model(inputs=inputs, outputs=output)

def custom_loss(rewards):
    def loss_fn(y_true, y_pred):
        
        cross_entropy_loss = tf.keras.losses.categorical_crossentropy(y_true, y_pred)
        weighted_loss = tf.multiply(rewards, cross_entropy_loss)
        return tf.reduce_mean(weighted_loss)
    return loss_fn

rewards = tf.constant([1.0, 0.5, 0.2])

model.compile(optimizer='adam', loss=custom_loss(rewards))

model.fit_generator(generator(file_list_numpy, file_list_midi, 5),steps_per_epoch=10, epochs=100)

In [ ]:
def loss_fn(y_true, y_pred):
        
        cross_entropy_loss = tf.keras.losses.categorical_crossentropy(y_true, y_pred)
        weighted_loss = tf.multiply(rewards, cross_entropy_loss)
        return tf.reduce_mean(weighted_loss)

In [ ]:
input_shape = (1, 160000)
inputs = Input(shape=input_shape)

layer1 = LSTM(256, return_sequences=True)(inputs)
layer3 = Dense(4096, activation = 'relu')(layer1)
layer4 = Dropout(0.1)(layer3)
layer5 = Dense(4096, activation = 'sigmoid')(layer4)
layer6 = Dropout(0.1)(layer5)
layer7 = Dense(4096, activation = 'sigmoid')(layer6)
layer8 = Dropout(0.1)(layer7)
layer9 = Dense(4096, activation = 'relu')(layer8)

output = Dense(2502, activation='relu')(layer9)

model = Model(inputs=inputs, outputs=output)

def custom_loss(rewards):
    def loss_fn(y_true, y_pred):
        
        cross_entropy_loss = tf.keras.losses.categorical_crossentropy(y_true, y_pred)
        weighted_loss = tf.multiply(rewards, cross_entropy_loss)
        return tf.reduce_mean(weighted_loss)
    return loss_fn

rewards = tf.constant([1.0, 0.5, 0.2])

model.compile(optimizer='adam', loss=custom_loss(rewards))

model.fit_generator(generator(file_list_numpy, file_list_midi, 5),steps_per_epoch=10, epochs=3000)

In [ ]:
input_shape = (1, 160000)
inputs = Input(shape=input_shape)

layer1 = LSTM(256, return_sequences=True)(inputs)
output = Dense(2502, activation='relu')(layer1)

model = Model(inputs=inputs, outputs=output)

model.compile(optimizer='adam', loss='categorical_crossentropy')

model.fit_generator(generator(file_list_numpy, file_list_midi, 5),steps_per_epoch=10, epochs=10)

In [ ]:
model.save('model/model_docker')

In [ ]:
model = keras.models.load_model('model/lstm_dense_simgoid2_relu', custom_objects={'loss_fn': loss_fn})

In [ ]:
data_dir = "test_model"  # directory where your numpy arrays are located
file_list = os.listdir(data_dir)  # list all files in the directory
x_test = []

# loop over all files in the directory and load the numpy arrays
for file_name in file_list:
    if file_name.endswith(".npy"):  # check if the file is a numpy array
        file_path = os.path.join(data_dir, file_name)
        x_test.append(np.load(file_path).flatten())

x_test = np.array(x_test)
# Reshape x_train to have 3 dimensions
x_test = x_test.reshape( x_test.shape[0], -1, x_test.shape[1])
print(x_test.shape)

In [ ]:
result = model.predict(x_test)

result = np.array(result)
result = np.maximum(result.round(3),0)

pitch_output = np.array(result[0][2::3], dtype='int')
pitch_output = np.minimum(pitch_output,127)
start_time_output = result[0][0::3]
end_time_output = result[0][1::3]

print(start_time_output.shape)

In [ ]:
result = np.array(result)
result = np.maximum(result.round(3),0)

pitch_output = np.array(result[0][0][2::3], dtype='int')
pitch_output = np.minimum(pitch_output,127)
start_time_output = result[0][0][0::3]
end_time_output = result[0][0][1::3]

print(start_time_output.shape)

In [ ]:
pm = pretty_midi.PrettyMIDI(initial_tempo=80)
velocity = 100
inst = pretty_midi.Instrument(program=0, is_drum=False, name='piano')
pm.instruments.append(inst)
for i in range(len(pitch_output)):
    
    inst.notes.append(pretty_midi.Note(velocity, pitch_output[i], start_time_output[i], end_time_output[i]))


    
print('There are {} time signature changes'.format(len(pm.time_signature_changes)))
print('There are {} instruments'.format(len(pm.instruments)))
print('Instrument 1 has {} notes'.format(len(pm.instruments[0].notes)))
print('Instrument 1 has {} pitch bends'.format(len(pm.instruments[0].pitch_bends)))
print('Instrument 1 has {} control changes'.format(len(pm.instruments[0].control_changes)))

pm.write('outLstm6.mid')


In [ ]:
# load x
file_list_numpy = os.listdir('path')  # list all files in the directory
rewards = [0.1,0.8,0.6,1,0.6]
x_train = []

for i in range(len(rewards)):

    file_path = os.path.join('path', file_list_numpy[i])
    x_train.append(np.load(file_path).flatten())


def loss_fn(reward, y_pred):
    cross_entropy_loss = tf.keras.losses.BinaryCrossentropy(y_pred)
    weighted_loss = tf.multiply(reward, cross_entropy_loss)
    return tf.reduce_mean(weighted_loss)

# load the model with the custom loss
model = load_model('model/lstm_dense_simgoid2_relu', custom_objects={'loss_fn': loss_fn})

# compile the model with the custom loss function
model.compile(optimizer='adam', loss=loss_fn)

# now you can train the model
model.fit(x_train, rewards, epoch=len(rewards))
